Using kernel `conda_pytorch_latest_p36`

In [1]:
import sys
sys.path.append('../../../')

In [2]:
from pathlib import Path
import os
import random

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch

In [5]:
from deep.constants import *

## Data

In [6]:
data = FRAMEWORKS_PATH

In [7]:
afexportable = pd.read_csv(data /'afexportable_of_af_of_projects_of_interest.csv')
all_afs = pd.read_csv(data /'all_afs.csv')

proj_interest = pd.read_csv(data / 'projects_of_interest.csv')
entr_proj_interest = pd.read_csv(data / 'entries_of_projects_of_interest.csv')
exp_proj_interest = pd.read_csv(data / 'exportdata_of_entries_of_projects_of_interest.csv')
wid_proj_interest = pd.read_csv(data / 'widgets_of_afs_of_interest.csv')

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
train = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_train.csv')
val = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_val.csv')
test = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_test.csv')

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
dim = train[DIMENSION_CLASSES]

In [26]:
dim.describe()

,Shock Informaton,Effects Systems And Networks,Effects On Population,Capacities & Response,At Risk,Scope & Scale,Impact,Humanitarian Conditions,Priorities,Context
count,85922.000000,85922.000000,85922.000000,85922.000000,85922.000000,85922.000000,85922.000000,85922.000000,85922.000000,85922.000000
mean,0.022753,0.040839,0.048451,0.154908,0.043539,0.047508,0.186134,0.476642,0.074323,0.116315
std,0.149117,0.197919,0.214718,0.361819,0.204069,0.212724,0.389217,0.499457,0.262298,0.320604
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
train.shape

(85922, 46)

In [25]:
train.excerpt

0        Our concern for the impact of the pandemic on ...
1        WHO has provided sanitizers, PPE, and 92 infra...
2        For COVID-19: • Another batch of supplies, und...
3        On 28 April, the first repatriation flight of ...
4        UNICEF has further provided additional support...
                               ...                        
85917    Kismayo IDPs registered a GAM prevalence of 12...
85918    The poor nutrition situation in Kismayo IDPs i...
85919    Kismayo Urban recorded a GAM prevalence of 12....
85920    Galkacyo IDPs recorded a GAM prevalence of 10....
85921    As of 7 April, Somalia has recorded close to 1...
Name: excerpt, Length: 85922, dtype: object

In [22]:
train.columns

Index(['entry_id', 'excerpt', 'analysis_framework_id', 'created_by_id',
       'lead_id', 'modified_by_id', 'information_date', 'order', 'client_id',
       'project_id', 'tabular_field_id', 'dropped_excerpt', 'verified',
       'verification_last_changed_by_id', 'exportdata_id', 'exportable_id',
       'dimensions', 'subdimensions', 'sectors', 'subsectors', 'Shelter',
       'Livelihoods', 'Protection', 'Shelter and NFIs', 'NFI', 'Food Security',
       'Logistics', 'Cross', 'CCCM', 'Agriculture', 'Health', 'Nutrition',
       'Education', 'WASH', 'sector_ids', 'Shock Informaton',
       'Effects Systems And Networks', 'Effects On Population',
       'Capacities & Response', 'At Risk', 'Scope & Scale', 'Impact',
       'Humanitarian Conditions', 'Priorities', 'Context', 'dimension_ids'],
      dtype='object')

In [24]:
train['subdimensions']

0                        ['Physical & Mental Wellbeing']
1                                      ['International']
2                            ['National & Local Actors']
3                     ['Government & Local Authorities']
4                                      ['International']
                              ...                       
85917                           ['Humanitarian Profile']
85918                           ['Humanitarian Profile']
85919                                                 []
85920                                                 []
85921    ['Primary Shock (Covid Event Characteristics)']
Name: subdimensions, Length: 85922, dtype: object

In [ ]:
def process_for_sector(df, sector, train):
    relevant_train = df[df.is_relevant == 1]
    relevant_train.sector_ids = relevant_train.sector_ids.apply(eval)
    relevant_train = relevant_train[relevant_train.sector_ids.apply(len) > 0]
    
    positive_train = relevant_train[relevant_train.sector_ids.apply(lambda x: sector in x)]
    negative_train = relevant_train[relevant_train.sector_ids.apply(lambda x: sector not in x)]

    positive_train.sector_ids = 1
    negative_train.sector_ids = 0
    train_df = pd.concat([positive_train, negative_train])
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    
    train_df['is_valid'] = False if train else True
        
    return train_df

In [ ]:
train_df = process_for_sector(train, 4, True)
test_df = process_for_sector(test, 4, False)

In [ ]:
train_df

In [ ]:
train_df.to_pickle('train_df.pickle')
test_df.to_pickle('test_df.pickle')

## Sagemaker Prep

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = SAGEMAKER_BUCKET
prefix = "huggingface/first"  # Replace with the prefix under which you want to store the data if needed


### Bucket upload

In [ ]:
bucket_path = 'test1/data'
train_channel = bucket_path + "/train_df.pickle"
validation_channel = bucket_path + "/test_df.pickle"

sess.upload_data(path="train_df.pickle", bucket=SAGEMAKER_BUCKET, key_prefix=bucket_path)
sess.upload_data(path="test_df.pickle", bucket=SAGEMAKER_BUCKET, key_prefix=bucket_path)

s3_train_data = f"s3://{SAGEMAKER_BUCKET}/{train_channel}"
s3_validation_data = f"s3://{SAGEMAKER_BUCKET}/{validation_channel}"

s3_output_location = f"s3://{SAGEMAKER_BUCKET}/{bucket_path}/output"

In [ ]:
from sagemaker.huggingface import HuggingFace


# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name': 'distilbert-base-uncased'
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir=str(SCRIPTS_MODELS_PATH / 'stefano'),
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.4',
        pytorch_version='1.6',
        py_version='py36',
        hyperparameters = hyperparameters
)

In [ ]:
fit_arguments = {
    'train': f's3://{SAGEMAKER_BUCKET}/{bucket_path}',
    'test': f's3://{SAGEMAKER_BUCKET}/{bucket_path}'
}

In [ ]:
huggingface_estimator.fit(fit_arguments)

In [ ]:
1

## Train

In [ ]:
region_name = boto3.Session().region_name

In [ ]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print("Using SageMaker BlazingText container: {} ({})".format(container, region_name))

In [ ]:
bt_model = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.c4.4xlarge",
    volume_size=30,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    hyperparameters={
        "mode": "supervised",
        "epochs": 4,
        "min_count": 2,
        "learning_rate": 0.05,
        "vector_dim": 10,
#         "early_stopping": False,
        "patience": 4,
#         "min_epochs": 5,
#         "word_ngrams": 2,
    },
)

In [ ]:
# Create a train data channel with S3_data_type as 'AugmentedManifestFile' and attribute names.
# train_data = sagemaker.session.s3_input(
#     your_augmented_manifest_file,
#     distribution='FullyReplicated',
#     content_type='application/x-recordio',
#     s3_data_type='AugmentedManifestFile',
#     attribute_names=['source-ref', 'annotations'],
#     input_mode='Pipe',
#     record_wrapping='RecordIO'
# )

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="text/plain",
    s3_data_type="AugmentedManifestFile",
    input_mode='Pipe',
    attribute_names=['source', 'label'],
    record_wrapping='RecordIO',
)
validation_data = sagemaker.inputs.TrainingInput(
    s3_validation_data,
    distribution="FullyReplicated",
    content_type="text/plain",
    s3_data_type="AugmentedManifestFile",
    input_mode='Pipe',
    attribute_names=['source', 'label'],
    record_wrapping='RecordIO',
)
data_channels = {"train": train_data, "validation": validation_data}

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

In [ ]:
1

In [ ]:
from sagemaker.serializers import JSONSerializer

text_classifier = bt_model.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=JSONSerializer()
)


In [ ]:
sentences = list(test_df.sentence_text)

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [" ".join(nltk.word_tokenize(sent)) for sent in sentences]

payload = {"instances": tokenized_sentences, "configuration": {"k": 5}}

response = text_classifier.predict(payload)

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

In [ ]:
t = test_df.copy()

In [ ]:
a = []
thresh = 0.05
for pred in predictions:
    labels = [int(x.replace('__label__', '')) for x, y in zip(pred['label'], pred['prob']) if y > thresh]
    a.append(labels)

In [ ]:
t['preds'] = a

In [ ]:
indexes = []
recalls = []
precisions = []
f1_scores = []

for i, class_ in enumerate(classes):
    class_preds = [1 if i in x else 0 for x in t.preds]
    class_targets = [1 if i in x else 0 for x in t.sector_ids]

    indexes.append(class_)
    precisions.append(precision_score(class_targets, class_preds))
    recalls.append(recall_score(class_targets, class_preds))    
    f1_scores.append(f1_score(class_targets, class_preds))        


all_metrics = pd.DataFrame(
    {
        'class': indexes,
        'recall': recalls,
        'precision': precisions,
        'f1_score': f1_scores
    }
).set_index('class', drop=True)

In [ ]:
all_metrics.plot(
    figsize=(20, 10), xticks=range(12), yticks=[x/10 for x in range(11)], ylim=(0, 1), grid=True
)

In [ ]:
t